In [1]:
import pandas as pd
from sklearn import metrics, preprocessing
from operator import itemgetter

from IPython.display import display

In [2]:
df = pd.read_csv('/mnt/isilon/cbmi/variome/perry/projects/sarmadi/mahdi_epi/work/roc_df_panel/ccr-vest-missense_badness', sep='\t')
df.head()

,# of probands,Disease,PredictionStatus,Total,Unique,alt,ccr,chrom,class,clin_class,...,is_domain,missense_badness,missense_depletion,pfam,pos,ref,vest,y,ccr-vest-missense_badness_pred_lm,ccr-vest-missense_badness_probaPred
0,NaN,EPI,CorrectBenign,NaN,NaN,A,0.0000,7,B,LIKELY_BENIGN,...,0,0.3248,-1.0866,none116950,145814041,G,0.421,0,0,0.133809
1,NaN,EPI,CorrectBenign,NaN,NaN,T,3.0801,7,B,LIKELY_BENIGN,...,0,0.3523,-1.0866,none116950,145814051,C,0.327,0,0,0.101873
2,NaN,EPI,CorrectBenign,NaN,NaN,T,0.0000,7,B,LIKELY_BENIGN,...,0,0.5057,-1.0866,none116950,146471398,C,0.777,0,0,0.366999
3,NaN,EPI,CorrectBenign,NaN,NaN,A,0.0000,7,B,LIKELY_BENIGN,...,0,0.3599,-1.0866,none116950,146471401,G,0.055,0,0,0.031341
4,NaN,EPI,CorrectBenign,NaN,NaN,C,0.0000,7,B,LIKELY_BENIGN,...,1,0.4317,-1.0866,"Laminin_G_1:271,Laminin_G_2:389",146818170,G,0.438,0,0,0.130675


In [11]:
df.index

RangeIndex(start=0, stop=959, step=1)

In [3]:
cols = ['ccr', 'vest', 'missense_badness']
x = preprocessing.scale(df[cols])
print(x)

[[-0.82544521 -0.35685767 -0.71161946]
 [-0.75429086 -0.63348996 -0.5695537 ]
 [-0.82544521  0.69081354  0.22291489]
 ..., 
 [ 1.46280658  0.94684555  1.96696571]
 [-0.82544521 -1.07492446 -1.00195019]
 [-0.82544521 -1.37215702 -0.5695537 ]]


In [9]:
poly = preprocessing.PolynomialFeatures(2, interaction_only=True, include_bias=False)
x = poly.fit_transform(df[cols])
print(poly.get_feature_names())

['x0', 'x1', 'x2', 'x0 x1', 'x0 x2', 'x1 x2']


In [14]:
poly = preprocessing.PolynomialFeatures(2, interaction_only=True, include_bias=False)
x = preprocessing.scale( poly.fit_transform(df[cols]) )
df_trans = pd.DataFrame(x, index=df.index, columns=poly.get_feature_names())
final_df = pd.merge(df_trans, df, right_index=True, left_index=True)
final_df.head()

,x0,x1,x2,x0 x1,x0 x2,x1 x2,# of probands,Disease,PredictionStatus,Total,...,is_domain,missense_badness,missense_depletion,pfam,pos,ref,vest,y,ccr-vest-missense_badness_pred_lm,ccr-vest-missense_badness_probaPred
0,-0.825445,-0.356858,-0.711619,-0.759732,-0.733080,-0.594379,NaN,EPI,CorrectBenign,NaN,...,0,0.3248,-1.0866,none116950,145814041,G,0.421,0,0,0.133809
1,-0.754291,-0.633490,-0.569554,-0.732951,-0.691532,-0.682943,NaN,EPI,CorrectBenign,NaN,...,0,0.3523,-1.0866,none116950,145814051,C,0.327,0,0,0.101873
2,-0.825445,0.690814,0.222915,-0.759732,-0.733080,0.459032,NaN,EPI,CorrectBenign,NaN,...,0,0.5057,-1.0866,none116950,146471398,C,0.777,0,0,0.366999
3,-0.825445,-1.433958,-0.530292,-0.759732,-0.733080,-1.075246,NaN,EPI,CorrectBenign,NaN,...,0,0.3599,-1.0866,none116950,146471401,G,0.055,0,0,0.031341
4,-0.825445,-0.306828,-0.159371,-0.759732,-0.733080,-0.379148,NaN,EPI,CorrectBenign,NaN,...,1,0.4317,-1.0866,"Laminin_G_1:271,Laminin_G_2:389",146818170,G,0.438,0,0,0.130675


In [4]:
df[cols].head()

,ccr,vest,missense_badness
0,0.0000,0.421,0.3248
1,3.0801,0.327,0.3523
2,0.0000,0.777,0.5057
3,0.0000,0.055,0.3599
4,0.0000,0.438,0.4317


In [3]:
auc_ls = []
scores = []
feats = ['ccr', 'vest', 'fathmm', 'missense_badness', 'missense_depletion']
def eval_roc(df, disease):
    scores = [x for x in df.columns.values if '_probaPred' in x or x in feats]
    for score in scores:
        fpr, tpr, _ = metrics.roc_curve(df['y'], df[score], pos_label=1)
        auc = metrics.auc(fpr, tpr)
        avg_pr = metrics.average_precision_score(df['y'], df[score])
        auc_ls.append((auc, avg_pr, score, disease))

diseases = set(df['Disease'])
#with open('roc_eval', 'w') as fout:
#    print('disease\tfpr\ttpr\tfeature', file=fout)
for dis in diseases:
    eval_roc(df[dis==df.Disease], dis)
auc_df = pd.DataFrame(auc_ls, columns=['auc', 'avg_pr', 'features', 'disease'])
#sorted(auc_ls, key=itemgetter(0))

In [4]:
for disease in diseases:
    display( auc_df[auc_df.disease==disease].sort_values(by='avg_pr', ascending=False) )

,auc,avg_pr,features,disease
0,0.948013,0.968505,ccr,genedx-epi-limitGene
4,0.903606,0.928870,vest,genedx-epi-limitGene
5,0.920343,0.900993,ccr-vest-missense_badness_probaPred,genedx-epi-limitGene
3,0.774413,0.838421,missense_depletion,genedx-epi-limitGene
1,0.847233,0.824411,fathmm,genedx-epi-limitGene
2,0.677728,0.772449,missense_badness,genedx-epi-limitGene


,auc,avg_pr,features,disease
6,0.882353,0.922670,ccr,Rasopathies
10,0.882353,0.899949,vest,Rasopathies
11,0.878431,0.830028,ccr-vest-missense_badness_probaPred,Rasopathies
9,0.768627,0.732618,missense_depletion,Rasopathies
7,0.670588,0.713709,fathmm,Rasopathies
8,0.768627,0.708965,missense_badness,Rasopathies


,auc,avg_pr,features,disease
17,0.954710,0.922683,ccr-vest-missense_badness_probaPred,EPI
12,0.892630,0.914689,ccr,EPI
16,0.934586,0.912911,vest,EPI
13,0.843814,0.778806,fathmm,EPI
15,0.774805,0.768908,missense_depletion,EPI
14,0.663953,0.609898,missense_badness,EPI


,auc,avg_pr,features,disease
21,0.956506,0.809572,missense_depletion,Cardiomyopathy
18,0.902169,0.801626,ccr,Cardiomyopathy
23,0.900482,0.785806,ccr-vest-missense_badness_probaPred,Cardiomyopathy
22,0.840361,0.459964,vest,Cardiomyopathy
19,0.844699,0.394449,fathmm,Cardiomyopathy
20,0.610602,0.233763,missense_badness,Cardiomyopathy
